In [70]:
import os

In [71]:
roots=[]
meta_data = {'class_names':'','class_roots':''}
test_path = r'C:\Users\harsh\Downloads\images_background\images_background'

In [72]:
def extract_filenames(path):
    for name in os.listdir(test_path):
        roots.append((os.path.join(test_path,name)))
    meta_data['class_names'] = os.listdir(path)
    meta_data['class_roots'] = roots 

In [73]:
extract_filenames(test_path)

In [74]:
meta_data

{'class_names': ['Alphabet_of_the_Magi',
  'Anglo-Saxon_Futhorc',
  'Arcadian',
  'Armenian',
  'Asomtavruli_(Georgian)',
  'Balinese',
  'Bengali',
  'Blackfoot_(Canadian_Aboriginal_Syllabics)',
  'Braille',
  'Burmese_(Myanmar)',
  'Cyrillic',
  'Early_Aramaic',
  'Futurama',
  'Grantha',
  'Greek',
  'Gujarati',
  'Hebrew',
  'Inuktitut_(Canadian_Aboriginal_Syllabics)',
  'Japanese_(hiragana)',
  'Japanese_(katakana)',
  'Korean',
  'Latin',
  'Malay_(Jawi_-_Arabic)',
  'Mkhedruli_(Georgian)',
  'N_Ko',
  'Ojibwe_(Canadian_Aboriginal_Syllabics)',
  'Sanskrit',
  'Syriac_(Estrangelo)',
  'Tagalog',
  'Tifinagh'],
 'class_roots': ['C:\\Users\\harsh\\Downloads\\images_background\\images_background\\Alphabet_of_the_Magi',
  'C:\\Users\\harsh\\Downloads\\images_background\\images_background\\Anglo-Saxon_Futhorc',
  'C:\\Users\\harsh\\Downloads\\images_background\\images_background\\Arcadian',
  'C:\\Users\\harsh\\Downloads\\images_background\\images_background\\Armenian',
  'C:\\Users\\h

In [327]:
#file_details = {'filename':{'audio':'','frames':[]}}
#meta_data[0]['filename']['audio']
file_details={}

In [ ]:
def fewshotdata(path):
    for key,value in file_details.items():
        support_set = 

In [ ]:
import random
from typing import List, Tuple

import torch
from torch import Tensor
from torch.utils.data import Sampler

from easyfsl.datasets import FewShotDataset


class TaskSampler(Sampler):
    """
    Samples batches in the shape of few-shot classification tasks. At each iteration, it will sample
    n_way classes, and then sample support and query images from these classes.
    """

    def __init__(
        self,
        dataset: FewShotDataset,
        n_way: int,
        n_shot: int,
        n_query: int,
        n_tasks: int,
    ):
        """
        Args:
            dataset: dataset from which to sample classification tasks. Must have a field 'label': a
                list of length len(dataset) containing containing the labels of all images.
            n_way: number of classes in one task
            n_shot: number of support images for each class in one task
            n_query: number of query images for each class in one task
            n_tasks: number of tasks to sample
        """
        super().__init__(data_source=None)
        self.n_way = n_way
        self.n_shot = n_shot
        self.n_query = n_query
        self.n_tasks = n_tasks

        self.items_per_label = {}
        for item, label in enumerate(dataset.get_labels()):
            if label in self.items_per_label.keys():
                self.items_per_label[label].append(item)
            else:
                self.items_per_label[label] = [item]

    def __len__(self):
        return self.n_tasks

    def __iter__(self):
        for _ in range(self.n_tasks):
            yield torch.cat(
                [
                    # pylint: disable=not-callable
                    torch.tensor(
                        random.sample(
                            self.items_per_label[label], self.n_shot + self.n_query
                        )
                    )
                    # pylint: enable=not-callable
                    for label in random.sample(self.items_per_label.keys(), self.n_way)
                ]
            )

    def episodic_collate_fn(
        self, input_data: List[Tuple[Tensor, int]]
    ) -> Tuple[Tensor, Tensor, Tensor, Tensor, List[int]]:
        """
        Collate function to be used as argument for the collate_fn parameter of episodic
            data loaders.
        Args:
            input_data: each element is a tuple containing:
                - an image as a torch Tensor
                - the label of this image
        Returns:
            tuple(Tensor, Tensor, Tensor, Tensor, list[int]): respectively:
                - support images,
                - their labels,
                - query images,
                - their labels,
                - the dataset class ids of the class sampled in the episode
        """

        true_class_ids = list({x[1] for x in input_data})

        all_images = torch.cat([x[0].unsqueeze(0) for x in input_data])
        all_images = all_images.reshape(
            (self.n_way, self.n_shot + self.n_query, *all_images.shape[1:])
        )
        # pylint: disable=not-callable
        all_labels = torch.tensor(
            [true_class_ids.index(x[1]) for x in input_data]
        ).reshape((self.n_way, self.n_shot + self.n_query))
        # pylint: enable=not-callable

        support_images = all_images[:, : self.n_shot].reshape(
            (-1, *all_images.shape[2:])
        )
        query_images = all_images[:, self.n_shot :].reshape((-1, *all_images.shape[2:]))
        support_labels = all_labels[:, : self.n_shot].flatten()
        query_labels = all_labels[:, self.n_shot :].flatten()

        return (
            support_images,
            support_labels,
            query_images,
            query_labels,
            true_class_ids,
        )

In [3]:
import json
from pathlib import Path
from typing import List, Union, Set, Tuple, Callable

from PIL import Image

from easyfsl.datasets import FewShotDataset
from easyfsl.datasets.default_configs import default_transform, DEFAULT_IMAGE_FORMATS


class EasySet(FewShotDataset):
    """
    A ready-to-use dataset. Will work for any dataset where the images are
    grouped in directories by class. It expects a JSON file defining the
    classes and where to find them. It must have the following shape:
        {
            "class_names": [
                "class_1",
                "class_2"
            ],
            "class_roots": [
                "path/to/class_1_folder",
                "path/to/class_2_folder"
            ]
        }
    """

    def __init__(
        self,
        specs_file: Union[Path, str],
        image_size: int = 84,
        transform: Callable = None,
        training: bool = False,
        supported_formats: Set[str] = None,
    ):
        """
        Args:
            specs_file: path to the JSON file
            image_size: images returned by the dataset will be square images of the given size
            transform: torchvision transforms to be applied to images. If none is provided,
                we use some standard transformations including ImageNet normalization.
                These default transformations depend on the "training" argument.
            training: preprocessing is slightly different for a training set, adding a random
                cropping and a random horizontal flip. Only used if transforms = None.
            supported_formats: set of allowed file format. When listing data instances, EasySet
                will only consider these files. If none is provided, we use the default set of
                image formats.
        """
        specs = self.load_specs(Path(specs_file))

        self.images, self.labels = self.list_data_instances(
            specs["class_roots"], supported_formats=supported_formats
        )

        self.class_names = specs["class_names"]

        self.transform = (
            transform if transform else default_transform(image_size, training)
        )

    @staticmethod
    def load_specs(specs_file: Path) -> dict:
        """
        Load specs from a JSON file.
        Args:
            specs_file: path to the JSON file
        Returns:
            dictionary contained in the JSON file
        Raises:
            ValueError: if specs_file is not a JSON, or if it is a JSON and the content is not
                of the expected shape.
        """

        if specs_file.suffix != ".json":
            raise ValueError("EasySet requires specs in a JSON file.")

        with open(specs_file, "r") as file:
            specs = json.load(file)

        if "class_names" not in specs.keys() or "class_roots" not in specs.keys():
            raise ValueError(
                "EasySet requires specs in a JSON file with the keys class_names and class_roots."
            )

        if len(specs["class_names"]) != len(specs["class_roots"]):
            raise ValueError(
                "Number of class names does not match the number of class root directories."
            )

        return specs

    @staticmethod
    def list_data_instances(
        class_roots: List[str], supported_formats: Set[str] = None
    ) -> Tuple[List[str], List[int]]:
        """
        Explore the directories specified in class_roots to find all data instances.
        Args:
            class_roots: each element is the path to the directory containing the elements
                of one class
        Returns:
            list of paths to the images, and a list of same length containing the integer label
                of each image
        """
        if supported_formats is None:
            supported_formats = DEFAULT_IMAGE_FORMATS

        images = []
        labels = []
        for class_id, class_root in enumerate(class_roots):
            class_images = [
                str(image_path)
                for image_path in sorted(Path(class_root).glob("*"))
                if image_path.is_file() & (image_path.suffix in supported_formats)
            ]
            images += class_images
            labels += len(class_images) * [class_id]

        return images, labels

    def __getitem__(self, item: int):
        """
        Get a data sample from its integer id.
        Args:
            item: sample's integer id
        Returns:
            data sample in the form of a tuple (image, label), where label is an integer.
            The type of the image object depends of the output type of self.transform. By default
            it's a torch.Tensor, however you are free to define any function as self.transform, and
            therefore any type for the output image. For instance, if self.transform = lambda x: x,
            then the output image will be of type PIL.Image.Image.
        """
        # Some images of ILSVRC2015 are grayscale, so we convert everything to RGB for consistence.
        # If you want to work on grayscale images, use torch.transforms.Grayscale in your
        # transformation pipeline.
        img = self.transform(Image.open(self.images[item]).convert("RGB"))
        label = self.labels[item]

        return img, label

    def __len__(self) -> int:
        return len(self.labels)

    def get_labels(self) -> List[int]:
        return self.labels

    def number_of_classes(self):
        return len(self.class_names)

In [329]:
def generate_metadata(test_path):
    for name in os.listdir(test_path):
        file_path = os.path.join(test_path,name)
        for x,y,files in os.walk(file_path):
            frames = [file for file in files if file[-3:] == 'jpg']
            audio = [file for file in files if file[-3:] == 'wav']
            file_details[file_path]={'audio':audio,'frames':frames}

In [2]:
pip install easyfsl

     -------------------------------------- 49.2/49.2 KB 418.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [333]:
generate_metadata(test_path)
file_details

{'C:\\Users\\harsh\\Downloads\\images_evaluation\\images_evaluation\\Alphabet_of_the_Magi': {'audio': [],
  'frames': []},
 'C:\\Users\\harsh\\Downloads\\images_evaluation\\images_evaluation\\Angelic': {'audio': [],
  'frames': []},
 'C:\\Users\\harsh\\Downloads\\images_evaluation\\images_evaluation\\Anglo-Saxon_Futhorc': {'audio': [],
  'frames': []},
 'C:\\Users\\harsh\\Downloads\\images_evaluation\\images_evaluation\\Arcadian': {'audio': [],
  'frames': []}}

In [334]:
for key,value in file_details.items():
    print(key[],':',value)
    #print(key,':',value['audio'])

C:\Users\harsh\Downloads\images_evaluation\images_evaluation\Alphabet_of_the_Magi : {'audio': [], 'frames': []}
C:\Users\harsh\Downloads\images_evaluation\images_evaluation\Angelic : {'audio': [], 'frames': []}
C:\Users\harsh\Downloads\images_evaluation\images_evaluation\Anglo-Saxon_Futhorc : {'audio': [], 'frames': []}
C:\Users\harsh\Downloads\images_evaluation\images_evaluation\Arcadian : {'audio': [], 'frames': []}


In [304]:
test_path = r'C:\Users\harsh\Downloads\images_evaluation\images_evaluation'